In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model

FTEST = './data/test/test_labels.csv'

TEST_IMAGE_PATH = './data/test/'

model_path = "output_model/Hand_Location_CNN_Model.h5"

In [ ]:
IMG_H = 720
IMG_W = 1280


BREAK_POINT = 1

In [ ]:
fname = FTEST
df = pd.read_csv(os.path.expanduser(fname))

In [ ]:
img_num = 0
x_test = []

for img_path in df['filename']:
    #print(img_label_path)
    img_num += 1
    
    img_path = TEST_IMAGE_PATH + img_path
    # print("     ", picture)
    image = cv2.imread(img_path, 0)  # open .jpg data
    # print("     ", np.array(image).shape)
    image = np.array(image).astype(np.float32)
    image = image / 255  # normalize in 0 ~ 1
    x_test.append(image)
    
    # check BREAK_POINT
    if img_num == BREAK_POINT:
        break
    

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = x_test.reshape(1, IMG_H, IMG_W, 1)

In [ ]:
x_test.shape

In [ ]:
model = load_model(model_path)

In [ ]:
result = model.predict(x_test)

In [ ]:
result

In [ ]:
x_min = int(result[0][0] * IMG_W)
x_min

In [ ]:
y_min = int(result[0][1] * IMG_H)
y_min

In [ ]:
x_max = int(result[0][2] * IMG_W)
x_max

In [ ]:
y_max = int(result[0][3] * IMG_H)
y_max

In [ ]:
for img_path in df['filename']:
    #print(img_label_path)
    img_num += 1
    
    img_path = TEST_IMAGE_PATH + img_path
    # print("     ", picture)
    image = cv2.imread(img_path, 1)  # open .jpg data
   
    
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('image', image)

    cv2.waitKey(0)
    # check BREAK_POINT
    if img_num == BREAK_POINT:
        break

# Test with Real-time Camera

In [ ]:
# camera (input) configuration
frame_in_w = 1280
frame_in_h = 720
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
print("capture device is open: " + str(videoIn.isOpened()))

In [ ]:
while True:
    # keyboard input value
    key = cv2.waitKey(1) & 0xFF

    # read video from camera
    ret, outframe = videoIn.read()
    if (ret):

        gray = cv2.cvtColor(outframe, cv2.COLOR_BGR2GRAY)
        image = np.array(gray).astype(np.float32)
        image = image / 255  # normalize in 0 ~ 1
        
        x_test = np.array(x_test)
        x_test = x_test.reshape(1, 1, IMG_H, IMG_W,)
        
        result = model.predict(x_test)
        
        
        x_min = int(result[0][0] * IMG_W)
        y_min = int(result[0][1] * IMG_H)
        x_max = int(result[0][2] * IMG_W)
        y_max = int(result[0][3] * IMG_H)
        
        cv2.rectangle(outframe, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('outframe', outframe)

    else:
        raise RuntimeError("Error while reading from camera.")

    if key == ord('q'):     # press 'q' to leave while
        break

print('Video Capture end, release camera.')
videoIn.release()
cv2.destroyAllWindows()